In [1]:
# import all the admission, exit and living files

import pandas as pd
import glob

#admission

path = 'K:/Ad Hoc/AD Hoc Projects/May 2020/Annalect/Files/Admissions/Admissions' # use your path
all_files = glob.glob(path + "/*.csv")

li1 = []

for filename1 in all_files:
    df1 = pd.read_csv(filename1, index_col=None, header=0, encoding= 'unicode_escape')
    li1.append(df1)

admissions = pd.concat(li1, axis=0, ignore_index=True)

#exit

path = 'K:/Ad Hoc/AD Hoc Projects/May 2020/Annalect/Files/Exits/Exits' # use your path
all_files = glob.glob(path + "/*.csv")

li2 = []

for filename2 in all_files:
    df2 = pd.read_csv(filename2, index_col=None, header=0, encoding= 'unicode_escape')
    li2.append(df2)

exits = pd.concat(li2, axis=0, ignore_index=True)



#living



path = 'K:/Ad Hoc/AD Hoc Projects/May 2020/Annalect/Files/Living/Living' # use your path
all_files = glob.glob(path + "/*.csv")

li3 = []

for filename3 in all_files:
    df3 = pd.read_csv(filename3, index_col=None, header=0, encoding= 'unicode_escape')
    li3.append(df3)

living = pd.concat(li3, axis=0, ignore_index=True)

C:\Users\ullahm\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\ullahm\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#intial check
#admissions.head()
#exits.head()
#living.head()
#admissions.info()

In [3]:
# Sum up the counts of records and filter only residential care

#admissions
admissions['Admission']='Y'
admissions=admissions[admissions['CARE_TYPE']=="Residential care"]
admissions1=admissions[['YEAR','STATE','ACPR_NAME','AGE_GROUP','SEX','INDIGENOUS_STATUS','Admission']].copy()
AD=admissions1.groupby(['YEAR','STATE','ACPR_NAME','AGE_GROUP','SEX','INDIGENOUS_STATUS'])["Admission"].count().reset_index()

#exits
exits['Exit']='Y'
exits=exits[exits['CARE_TYPE']=="Residential care"]
exits1=exits[['YEAR','STATE','ACPR_NAME','AGE_GROUP','SEX','INDIGENOUS_STATUS','Exit']].copy()
exits2=exits[['YEAR','STATE','ACPR_NAME','AGE_GROUP','SEX','INDIGENOUS_STATUS','LOS_MONTHS']].copy()
E1=exits1.groupby(['YEAR','STATE','ACPR_NAME','AGE_GROUP','SEX','INDIGENOUS_STATUS'])["Exit"].count().reset_index()
E2=exits2.groupby(['YEAR','STATE','ACPR_NAME','AGE_GROUP','SEX','INDIGENOUS_STATUS'])['LOS_MONTHS'].mean().reset_index()
E=pd.merge(E1, E2, how='inner', on=['YEAR','STATE','ACPR_NAME','AGE_GROUP','SEX','INDIGENOUS_STATUS'])

#living
living['Living']='Y'
living=living[living['CARE_TYPE']=="Residential care"]
living1=living[['YEAR','STATE','ACPR_NAME','AGE_GROUP','SEX','INDIGENOUS_STATUS','Living']].copy()
L=living1.groupby(['YEAR','STATE','ACPR_NAME','AGE_GROUP','SEX','INDIGENOUS_STATUS'])["Living"].count().reset_index()

In [4]:
# data exploration

#AD.head()
#E.head()
#L.head()
#AD['AGE_GROUP'].value_counts()
#E['AGE_GROUP'].value_counts()
#L['CARE_TYPE'].value_counts()
#E['CARE_TYPE'].value_counts()
#AD['CARE_TYPE'].value_counts()

In [5]:
#Merge all the data frames
import numpy as np
from functools import reduce
data_frames = [AD, E, L]

nan_value = 0

data = reduce(lambda  left,right: pd.merge(left,right,on=['YEAR','STATE','ACPR_NAME','AGE_GROUP','SEX','INDIGENOUS_STATUS'],how='outer'), data_frames)
data.replace(np.nan, 0, inplace=True)

In [6]:
#Take out dataset for Dashboarding
data.to_csv('K:/Ad Hoc/AD Hoc Projects/May 2020/Annalect/Files/all.csv', encoding='utf-8', index=False)

In [48]:
# Import back the file and the places file
import pandas as pd


file1='K:/Ad Hoc/AD Hoc Projects/May 2020/Annalect/Files/all.csv'
file2='K:/Ad Hoc/AD Hoc Projects/May 2020/Annalect/Files/Places/ServicesPlaces_2008to2017_GENdata.csv'
Data=pd.read_csv(file1, index_col=None, header=0, encoding= 'unicode_escape')
Places=pd.read_csv(file2, index_col=None, header=0, encoding= 'unicode_escape')

In [49]:
# Transpose all categorical fields
from functools import reduce

#Admissions

Data['Age Group A'] = 'A' + Data['AGE_GROUP']
Data['SEX A']='A' + Data['SEX']
Data['A_I']='A'+Data['INDIGENOUS_STATUS'].astype(str)

data1=pd.pivot_table(Data, values='Admission', index=['YEAR','STATE','ACPR_NAME'], columns='Age Group A').reset_index()
data2=pd.pivot_table(Data, values='Admission', index=['YEAR','STATE','ACPR_NAME'], columns='SEX A').reset_index()
data3=pd.pivot_table(Data, values='Admission', index=['YEAR','STATE','ACPR_NAME'], columns='A_I').reset_index()

data=[data1,data2,data3]
data_a=reduce(lambda  left,right: pd.merge(left,right,on=['YEAR','STATE','ACPR_NAME'],how='inner'), data)

#Exits

Data['Age Group E'] = 'E' + Data['AGE_GROUP']
Data['SEX E']='E' + Data['SEX']
Data['E_I']='E'+Data['INDIGENOUS_STATUS'].astype(str)

data1=pd.pivot_table(Data, values='Exit', index=['YEAR','STATE','ACPR_NAME'], columns='Age Group E').reset_index()
data2=pd.pivot_table(Data, values='Exit', index=['YEAR','STATE','ACPR_NAME'], columns='SEX E').reset_index()
data3=pd.pivot_table(Data, values='Exit', index=['YEAR','STATE','ACPR_NAME'], columns='E_I').reset_index()

data=[data1,data2,data3]
data_e=reduce(lambda  left,right: pd.merge(left,right,on=['YEAR','STATE','ACPR_NAME'],how='inner'), data)

#Living

Data['Age Group L'] = 'L' + Data['AGE_GROUP']
Data['SEX L']='L' + Data['SEX']
Data['E_L']='L'+Data['INDIGENOUS_STATUS'].astype(str)

data1=pd.pivot_table(Data, values='Living', index=['YEAR','STATE','ACPR_NAME'], columns='Age Group L').reset_index()
data2=pd.pivot_table(Data, values='Living', index=['YEAR','STATE','ACPR_NAME'], columns='SEX L').reset_index()
data3=pd.pivot_table(Data, values='Living', index=['YEAR','STATE','ACPR_NAME'], columns='E_L').reset_index()

data=[data1,data2,data3]
data_l=reduce(lambda  left,right: pd.merge(left,right,on=['YEAR','STATE','ACPR_NAME'],how='inner'), data)

#Remerge all grouped values

All=[data_a,data_e,data_l]
df=reduce(lambda  left,right: pd.merge(left,right,on=['YEAR','STATE','ACPR_NAME'],how='inner'), All)

In [50]:
# Merge Places file with summarised admission,exits and living file

Places=Places[Places['PROGRAM_TYPE']=="Residential care"]
p=Places[['YEAR','STATE','ACPR_NAME','OPERATIONAL_PLACES']].copy()
p=p.groupby(['YEAR','STATE','ACPR_NAME'])['OPERATIONAL_PLACES'].sum().reset_index()
df=pd.merge(p,df,on=['YEAR','STATE','ACPR_NAME'],how='inner')

#log all numeric variables to get % change
import numpy as np
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
for c in [c for c in df.columns if df[c].dtype in numerics]:
    df[c] = np.where(df[c]>0,np.log10(df[c]),0)

In [51]:
# Create % of all admission , exit and living data with the operational places

cols = [each for each in df.columns if each not in ('YEAR','STATE','ACPR_NAME','OPERATIONAL_PLACES')]

df[cols] = df.apply(lambda x:  x[cols]/x['OPERATIONAL_PLACES'], axis=1)

# Log the operation places column to get % change

df["OPERATIONAL_PLACES"]=np.where(df['OPERATIONAL_PLACES']>0, np.log(df['OPERATIONAL_PLACES']), 0)

#Plot Distributions

%matplotlib inline
import matplotlib.pyplot as plt
df.hist(bins=50, figsize=(20,15))

plt.show()

#Using Pearson Correlation
# Too many fields to do correl matrix heatmap

# Correlation 
corr_matrix = df.corr()
corr_matrix["OPERATIONAL_PLACES"].sort_values(ascending=False)

In [52]:
#df.head()
df.reset_index()

,index,YEAR,STATE,ACPR_NAME,OPERATIONAL_PLACES,A0-49,A100+,A50-54,A55-59,A60-64,...,L75-79,L80-84,L85-89,L90-94,L95-99,LF,LM,L1,L2,L3
0,0,2008,ACT,ACT,7.491645,0.001255,0.000744,0.000697,0.001301,0.005020,...,0.013176,0.026850,0.041272,0.030210,0.013274,0.017040,0.010318,0.000186,0.020140,0.000644
1,1,2008,NSW,Central Coast,8.142063,0.000757,0.000970,0.000790,0.001513,0.002037,...,0.011738,0.025357,0.022463,0.020793,0.009653,0.014138,0.007171,0.000112,0.019658,0.000407
2,2,2008,NSW,Central West,7.363280,0.001268,0.000951,0.000951,0.001480,0.002642,...,0.016328,0.021419,0.028958,0.023191,0.011795,0.017446,0.007483,0.000272,0.020168,0.000770
3,3,2008,NSW,Far North Coast,8.006034,0.000467,0.000778,0.000467,0.001400,0.002584,...,0.014797,0.021155,0.023091,0.025133,0.009336,0.017637,0.006560,0.000185,0.020253,0.000731
4,4,2008,NSW,Hunter,8.597482,0.000431,0.001046,0.000267,0.000992,0.001532,...,0.012202,0.021650,0.024833,0.020590,0.010028,0.012823,0.005990,0.000092,0.020708,0.000458
5,5,2008,NSW,Illawarra,8.169053,0.000567,0.000661,0.000637,0.000992,0.001877,...,0.015156,0.022978,0.025779,0.017092,0.007851,0.013997,0.006697,0.000195,0.018592,0.000890
6,6,2008,NSW,Inner West,8.422663,0.000659,0.000923,0.000797,0.000857,0.003053,...,0.012091,0.019565,0.022049,0.014924,0.007254,0.011134,0.006184,0.000153,0.019197,0.001246
7,7,2008,NSW,Mid North Coast,8.131825,0.001029,0.000196,0.000588,0.001213,0.001727,...,0.011467,0.017963,0.023442,0.020353,0.009654,0.013634,0.006175,0.000281,0.019393,0.000550
8,8,2008,NSW,Nepean,7.591357,0.001298,0.000126,0.001587,0.001178,0.002776,...,0.010050,0.020040,0.033127,0.020318,0.009086,0.013629,0.006941,0.000236,0.020214,0.001312
9,9,2008,NSW,New England,7.360104,0.001018,0.000636,0.001060,0.001272,0.001999,...,0.013995,0.019148,0.027000,0.022980,0.010072,0.014008,0.007543,0.000524,0.020215,0.000848


In [53]:
# Standardize all numeric variables

from sklearn.preprocessing import StandardScaler
num_cols = df.columns[df.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

C:\Users\ullahm\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\ullahm\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [54]:
# Random forrest regessor for feature importances


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso


df.replace(np.nan, 0, inplace=True)
df= df.drop('YEAR', axis = 1)
df= df.drop('STATE', axis = 1)
df= df.drop('ACPR_NAME', axis = 1)
df= df.drop('L1', axis = 1)
df1= df.drop('OPERATIONAL_PLACES',axis = 1)

features=df1.copy()
#features['Year'].astype(str)

# Labels are the values we want to predict
labels = np.array(df['OPERATIONAL_PLACES'])




# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)


# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 100 decision trees
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

In [55]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.15 degrees.


In [56]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: A1                   Importance: 0.59
Variable: E1                   Importance: 0.16
Variable: LF                   Importance: 0.09
Variable: L3                   Importance: 0.04
Variable: LM                   Importance: 0.02
Variable: L2                   Importance: 0.02
Variable: A70-74               Importance: 0.01
Variable: L55-59               Importance: 0.01
Variable: L60-64               Importance: 0.01
Variable: A0-49                Importance: 0.0
Variable: A100+                Importance: 0.0
Variable: A50-54               Importance: 0.0
Variable: A55-59               Importance: 0.0
Variable: A60-64               Importance: 0.0
Variable: A65-69               Importance: 0.0
Variable: A75-79               Importance: 0.0
Variable: A80-84               Importance: 0.0
Variable: A85-89               Importance: 0.0
Variable: A90-94               Importance: 0.0
Variable: A95-99               Importance: 0.0
Variable: AF                   Importance: 0.0
Vari

In [57]:
# Linear model OLS regression to get variable coeeficients that increase residential places in Health Regions

from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
# create linear regression object
Y=labels
df2=df1[['A70-74','AM','EM','L60-64','A1','E1','LF','LM','L2','L3']].copy()
X=np.array(df2)

model = LinearRegression()
model.fit(X, Y)
print(model)

# make predictions
expected = Y
predicted = model.predict(X)



# summarize the fit of the model
mse = np.mean((predicted-expected)**2)
print('Mean Absolute Error:', metrics.mean_absolute_error(expected, predicted))  
print('Mean Squared Error:', metrics.mean_squared_error(expected, predicted))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(expected, predicted)))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)
Mean Absolute Error: 0.3272307529176469
Mean Squared Error: 0.18213520260442426
Root Mean Squared Error: 0.4267730106326128


In [58]:
# Variable Coeff

f_list = list(df2.columns)
coeff_df = pd.DataFrame(model.coef_, f_list, columns=['Coefficient'])  
coeff_df

,Coefficient
A70-74,0.047761
AM,0.208750
EM,-0.207013
L60-64,0.204274
A1,0.151825
E1,-0.560359
LF,-0.312622
LM,-0.533793
L2,0.187268
L3,-0.224012


In [59]:
# Model With Census Data

living=Data[['STATE','ACPR_NAME','Living']].copy()
Exit=Data[['STATE','ACPR_NAME','Exit']].copy()
Admission=Data[['STATE','ACPR_NAME','Admission']].copy()
Places=Places[['STATE','ACPR_NAME','OPERATIONAL_PLACES']].copy()

L=living.groupby(['STATE','ACPR_NAME'])["Living"].sum().reset_index()
A=Admission.groupby(['STATE','ACPR_NAME'])["Admission"].sum().reset_index()
E=Exit.groupby(['STATE','ACPR_NAME'])["Exit"].sum().reset_index()
P=Places.groupby(['STATE','ACPR_NAME'])["OPERATIONAL_PLACES"].sum().reset_index()

dc=[L,A,E,P]
dc=reduce(lambda  left,right: pd.merge(left,right,on=['STATE','ACPR_NAME'],how='outer'), dc)

file='K:/Ad Hoc/AD Hoc Projects/May 2020/Annalect/Files/ABS/ABS Census.csv'
Census=pd.read_csv(file, index_col=None, header=0, encoding= 'unicode_escape')
df=pd.merge(dc,Census,on=['STATE','ACPR_NAME'],how='left')

In [60]:
#log all numeric variables to get % change
import numpy as np
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
for c in [c for c in df.columns if df[c].dtype in numerics]:
    df[c] = np.where(df[c]>0,np.log10(df[c]),0)

C:\Users\ullahm\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log10
  """


In [61]:
# Random forrest regessor for feature importances


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso


df.replace(np.nan, 0, inplace=True)

df= df.drop('STATE', axis = 1)
df= df.drop('ACPR_NAME', axis = 1)
df=df.drop('SA4 Region', axis=1)
df=df.drop('Living', axis=1)
df=df.drop('Admission', axis=1)
df=df.drop('Exit', axis=1)
df1= df.drop('OPERATIONAL_PLACES',axis = 1)


features=df1.copy()
#features['Year'].astype(str)

# Labels are the values we want to predict
labels = np.array(df['OPERATIONAL_PLACES'])




# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)


# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 100 decision trees
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

In [62]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.41 degrees.


In [63]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: Sum of No children   Importance: 0.24
Variable: Sum of Has need for assistance with core activities Importance: 0.19
Variable: Sum of Children      Importance: 0.14
Variable: Sum of Mortality Rate Importance: 0.13
Variable: Sum of Does not have need for assistance with core activities Importance: 0.11
Variable: Sum of Married       Importance: 0.1
Variable: Sum of Alone         Importance: 0.09


In [64]:
# Linear Model for Census

from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
# create linear regression object
Y=labels

X=features

model = LinearRegression()
model.fit(X, Y)
print(model)

# make predictions
expected = Y
predicted = model.predict(X)



# summarize the fit of the model
mse = np.mean((predicted-expected)**2)
print('Mean Absolute Error:', metrics.mean_absolute_error(expected, predicted))  
print('Mean Squared Error:', metrics.mean_squared_error(expected, predicted))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(expected, predicted)))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)
Mean Absolute Error: 0.3073237309576705
Mean Squared Error: 0.26897961230152545
Root Mean Squared Error: 0.5186324443201809


In [65]:
coeff_df = pd.DataFrame(model.coef_, feature_list, columns=['Coefficient'])  
coeff_df

,Coefficient
Sum of No children,1.409165
Sum of Children,5.763592
Sum of Married,2.453890
Sum of Alone,-1.530521
Sum of Has need for assistance with core activities,-1.867366
Sum of Does not have need for assistance with core activities,-5.625441
Sum of Mortality Rate,-0.220232
